<a href="https://colab.research.google.com/github/parkgunha/Chagawa_project/blob/main/%EC%84%A0%ED%98%84%EC%88%98_%EB%9E%9C%EB%8D%A4%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8_%ED%9A%8C%EA%B7%80_%EB%AA%A8%EB%8D%B8_%EC%B5%9C%EC%A0%81%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [4]:
import pandas as pd
file_path = '/content/drive/MyDrive/Colab Notebooks/full_2_data.csv'
full2_data =  pd.read_csv('full_2_data.csv')
full2_data

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,carMake
0,Octavia,2017,10550,Manual,25250,Petrol,150.0,54.3,1.4,skoda
1,Citigo,2018,8200,Manual,1264,Petrol,145.0,67.3,1.0,skoda
2,Octavia,2019,15650,Automatic,6825,Diesel,145.0,67.3,2.0,skoda
3,Yeti Outdoor,2015,14000,Automatic,28431,Diesel,165.0,51.4,2.0,skoda
4,Superb,2019,18350,Manual,10912,Petrol,150.0,40.9,1.5,skoda
...,...,...,...,...,...,...,...,...,...,...
99181,Fiesta,2016,7999,Manual,31348,Petrol,125.0,54.3,1.2,ford
99182,B-MAX,2017,8999,Manual,16700,Petrol,150.0,47.1,1.4,ford
99183,B-MAX,2014,7499,Manual,40700,Petrol,30.0,57.7,1.0,ford
99184,Focus,2015,9999,Manual,7010,Diesel,20.0,67.3,1.6,ford


In [5]:
#인코딩
from sklearn.preprocessing import LabelEncoder
import numpy as np
lable_list = ['model']
onhot_list = ['transmission', 'fuelType', 'carMake']

def Encoding(df, lable_list, onhot_list):
  """ 범주형데이터를 숫자형으로 변경하는 함수
  df: 변경할 데이터 프레임
  lable_list: 라벨 인코딩
  onhot_list: 원핫 인코딩
  """
  # 라벨 인코딩
  encoding_df=df.copy()
  le = LabelEncoder()
  encoding_df[lable_list] = encoding_df[lable_list].apply(le.fit_transform)
  # 원 핫 인코딩
  encoding_df = pd.get_dummies(encoding_df, columns=onhot_list, drop_first= True, dtype=float)
  # drop_first: 첫번째 더미 삭제, dtype: 불리언에서 정수형으로변경
  print(len(encoding_df.columns))
  return encoding_df

#데이터 분할
from sklearn.model_selection import train_test_split
def Data_split(df, price):
  """ 데이터를 분할하는 함수
  df: 분할할 데이터 프레임
  price: 종속 변수명
  출력
  X_train: 학습데이터
  X_test: 테스트 데이터
  y_train: 학습데이터(실제값)
  y_test: 테스트데이터(실제값)
  """
  X = df.drop(columns=price)
  y = df[price]

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
  print(X_train.shape, X_test.shape)
  print(y_train.shape, y_test.shape)
  return X_train, X_test, y_train, y_test

from mmap import mmap
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# 로그 변환
def Log_Trans(y_train, y_test):
  # y값 로그변환
  log_y_train = np.log1p(y_train)
  log_y_test = np.log1p(y_test)
  return log_y_train, log_y_test

# sd 스케일링
def Sd_Scaling(X_train, X_test, normalize_columns):
  #독립 변수 표준화(standscaler방법)
  sd_X_train = X_train.copy()
  sd_X_test = X_test.copy()
  standscaler = StandardScaler()
  #train데이터 스케일링
  sd_X_test = X_test.copy()
  sd_X_train[normalize_columns] = standscaler.fit_transform(sd_X_train[normalize_columns])
  #test데이터 스케일링
  sd_X_test[normalize_columns] = standscaler.transform(sd_X_test[normalize_columns])
  return sd_X_train, sd_X_test

def MinMax_Scaling(X_train, X_test, normalize_columns):
  #독립 변수 표준화(standscaler방법)
  mm_X_train = X_train.copy()
  mm_X_test = X_test.copy()
  minmaxscaler = MinMaxScaler()
  minmaxscaler.fit(X_train[normalize_columns])
  #train데이터 스케일링
  mm_X_train[normalize_columns] = minmaxscaler.transform(mm_X_train[normalize_columns])
  #test데이터 스케일링
  mm_X_test[normalize_columns] = minmaxscaler.transform(mm_X_test[normalize_columns])
  return mm_X_train, mm_X_test

#  표준 정규화
def Robust_Scaling(X_train, X_test, normalize_columns):
  #독립 변수 표준 정규화(RobustScaler방법)
  robust_X_train = X_train.copy()
  robust_X_test = X_test.copy()
  robustScaler = RobustScaler()
  #train데이터 스케일링
  robust_X_train[normalize_columns] = robustScaler.fit_transform(robust_X_train[normalize_columns])
  #test데이터 스케일링
  robust_X_test[normalize_columns] = robustScaler.transform(robust_X_test[normalize_columns])
  return robust_X_train, robust_X_test
from sklearn.ensemble import RandomForestRegressor

def Rfr_model(x1, x2, y1, y2):
  # 랜덤 포레스트 회귀 모델 학습
  X_train = x1 # X_train: 학습데이터
  X_test = x2 # X_test: 테스트 데이터
  y_train = y1 # y_train: 학습데이터(실제값)
  y_test = y2 # y_test: 테스트데이터(실제값)
  """
  출력
  rfr: 랜덤 포레스트 회귀 모델
  y_train_pred: 학습데이터 예측값
  y_test_pred: 테스트데이터 예측값
  """
  # 랜덤 포레스트 회귀 모델 학습
  rfr = RandomForestRegressor(random_state=42)
  rfr.fit(X_train, y_train)
  # Fitting된 모델로 예측 수행
  y_train_pred = rfr.predict(X_train)
  y_test_pred = rfr.predict(X_test)
  # 랜덤 포레스트 R2-score
  # 학습 정확도
  train_accuarcy = rfr.score(X_train, y_train)
  print("학습 정확도:",rfr.score(X_train, y_train))
  return rfr, y_train_pred, y_test_pred, train_accuarcy

def rfr_feature_importances(rfr, X_train):
  feature_names = X_train.columns
  importance_df = pd.DataFrame({
      'Feature': feature_names,
      'Importance': rfr.feature_importances_
  }).sort_values(by='Importance', ascending=False)
  print(importance_df.head())

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
import numpy as np

# 역변한 필요한 경우
def Exp_y(log_y_test, log_y_test_pred):
  trans_y_test = np.expm1(log_y_test) # 실제값
  trans_y_test_pred = np.expm1(log_y_test_pred) #예상값
  return trans_y_test, trans_y_test_pred

def model_evaluation(y_test, y_test_pred, result_name ) :
  """ 모델 평가 함수
  trans_y_test: 데스트 데이터 역변환 실제값
  trans_y_test_pred: 데스트데이터 역변환 예측값
  result_name: 결과를 저장할 컬럼 이름
  """
  mse = round(mean_squared_error(y_test, y_test_pred),3) # 실제 y값, 예측값
  mae = round(mean_absolute_error(y_test, y_test_pred),3)
  rmse = round(np.sqrt(mean_squared_error(y_test, y_test_pred)),3) # 실제 y값, 예측값
  r2 = round(r2_score(y_test, y_test_pred),3)
  mape = round((mean_absolute_percentage_error(y_test, y_test_pred)*100),3)

  print(f"\nLGBM {result_name} Results")
  print(f"평균 제곱 오차(MSE): {mse}")
  print(f"평균 절대 오차(MAE): {mae}")
  print(f"평균 제곱 오차(MSE): {rmse}")
  print(f"평균 절대비율 오차(MAPE): {mape}")
  print(f"결정 계수(R2): {r2}\n")

  result_list = ['mse', 'rmse', 'mae', 'mape', 'r2']
  result_name = str(result_name)
  result_df = pd.DataFrame(data=[mse, rmse, mae, mape, r2],
                           index=result_list, columns=[result_name])
  return result_df


In [6]:
# 연도 정규화 제외
df = full2_data.copy()
lable_list = ['model']
onhot_list = ['transmission', 'fuelType', 'carMake']
encoding_df = Encoding(df,lable_list,onhot_list)
X_train, X_test, y_train, y_test = Data_split(encoding_df, 'price')

normalize_columns = ["mileage", "tax", "mpg","engineSize"] # 표준 정규화할 변수 리스트

# 로그
log_y_train, log_y_test = Log_Trans(y_train, y_test)
log_rfr, y_train_pred, y_test_pred, train_accuarcy = Rfr_model(X_train, X_test, log_y_train, log_y_test)
trans_y_test, trans_y_test_pred = Exp_y(log_y_test, y_test_pred)
rfr_Log = model_evaluation(trans_y_test, trans_y_test_pred, result_name="rfr_Log")

# 정규화
log_y_train, log_y_test = Log_Trans(y_train, y_test)
mm_X_train, mm_X_test = MinMax_Scaling(X_train, X_test, normalize_columns)
mm_rfr, y_train_pred, y_test_pred, train_accuarcy = Rfr_model(mm_X_train, mm_X_test, log_y_train, log_y_test)
trans_y_test, trans_y_test_pred = Exp_y(log_y_test, y_test_pred)
rfr_mm = model_evaluation(trans_y_test, trans_y_test_pred, result_name="rfr_mm")

# 표준화
log_y_train, log_y_test = Log_Trans(y_train, y_test)
sd_X_train, sd_X_test = Sd_Scaling(X_train, X_test, normalize_columns)
sd_rfr, y_train_pred, y_test_pred, train_accuarcy = Rfr_model(sd_X_train, sd_X_test, log_y_train, log_y_test)
trans_y_test, trans_y_test_pred = Exp_y(log_y_test, y_test_pred)
rfr_sd = model_evaluation(trans_y_test, trans_y_test_pred, result_name="rfr_sd")

# 표준 정규화
log_y_train, log_y_test = Log_Trans(y_train, y_test)
robust_X_train, robust_X_test = Robust_Scaling(X_train, X_test, normalize_columns)
robust_rfr, y_train_pred, y_test_pred, train_accuarcy = Rfr_model(robust_X_train, robust_X_test, log_y_train, log_y_test)
trans_y_test, trans_y_test_pred = Exp_y(log_y_test, y_test_pred)
rfr_robust = model_evaluation(trans_y_test, trans_y_test_pred, result_name="rfr_robust")

result_df = pd.concat([rfr_Log, rfr_mm, rfr_sd, rfr_robust], axis=1)
pd.options.display.float_format = '{:.3f}'.format
result_df

22
(69430, 21) (29756, 21)
(69430,) (29756,)
학습 정확도: 0.9946784846225595

LGBM rfr_Log Results
평균 제곱 오차(MSE): 4996843.472
평균 절대 오차(MAE): 1175.527
평균 제곱 오차(MSE): 2235.362
평균 절대비율 오차(MAPE): 7.268
결정 계수(R2): 0.95

학습 정확도: 0.9946797469109543

LGBM rfr_mm Results
평균 제곱 오차(MSE): 4999095.81
평균 절대 오차(MAE): 1175.708
평균 제곱 오차(MSE): 2235.866
평균 절대비율 오차(MAPE): 7.269
결정 계수(R2): 0.95

학습 정확도: 0.9946811199475528

LGBM rfr_sd Results
평균 제곱 오차(MSE): 5001366.208
평균 절대 오차(MAE): 1175.832
평균 제곱 오차(MSE): 2236.373
평균 절대비율 오차(MAPE): 7.269
결정 계수(R2): 0.95

학습 정확도: 0.9946802406264056

LGBM rfr_robust Results
평균 제곱 오차(MSE): 4996276.151
평균 절대 오차(MAE): 1175.608
평균 제곱 오차(MSE): 2235.235
평균 절대비율 오차(MAPE): 7.268
결정 계수(R2): 0.95



,rfr_Log,rfr_mm,rfr_sd,rfr_robust
mse,4996843.472,4999095.810,5001366.208,4996276.151
rmse,2235.362,2235.866,2236.373,2235.235
mae,1175.527,1175.708,1175.832,1175.608
mape,7.268,7.269,7.269,7.268
r2,0.950,0.950,0.950,0.950


In [7]:
# 연도 이상치 1980이하 제거
df = full2_data.copy()
df = df[df["year"]>1980]
lable_list = ['model']
onhot_list = ['transmission', 'fuelType', 'carMake']
encoding_df = Encoding(df,lable_list,onhot_list)
X_train, X_test, y_train, y_test = Data_split(encoding_df, 'price')

normalize_columns = ["year","mileage", "tax", "mpg"] # 표준 정규화할 변수 리스트

# 로그
log_y_train, log_y_test = Log_Trans(y_train, y_test)
log_rfr, y_train_pred, y_test_pred, train_accuarcy = Rfr_model(X_train, X_test, log_y_train, log_y_test)
rfr_feature_importances(log_rfr, X_train)
trans_y_test, trans_y_test_pred = Exp_y(log_y_test, y_test_pred)
rfr_Log = model_evaluation(trans_y_test, trans_y_test_pred, result_name="rfr_Log")

# 정규화
log_y_train, log_y_test = Log_Trans(y_train, y_test)
mm_X_train, mm_X_test = MinMax_Scaling(X_train, X_test, normalize_columns)
mm_rfr, y_train_pred, y_test_pred, train_accuarcy = Rfr_model(mm_X_train, mm_X_test, log_y_train, log_y_test)
rfr_feature_importances(mm_rfr, mm_X_train)
trans_y_test, trans_y_test_pred = Exp_y(log_y_test, y_test_pred)
rfr_mm = model_evaluation(trans_y_test, trans_y_test_pred, result_name="rfr_mm")

# 표준화
log_y_train, log_y_test = Log_Trans(y_train, y_test)
sd_X_train, sd_X_test = Sd_Scaling(X_train, X_test, normalize_columns)
sd_rfr, y_train_pred, y_test_pred, train_accuarcy = Rfr_model(sd_X_train, sd_X_test, log_y_train, log_y_test)
rfr_feature_importances(sd_rfr, sd_X_train)
trans_y_test, trans_y_test_pred = Exp_y(log_y_test, y_test_pred)
rfr_sd = model_evaluation(trans_y_test, trans_y_test_pred, result_name="rfr_sd")

# 표준 정규화
log_y_train, log_y_test = Log_Trans(y_train, y_test)
robust_X_train, robust_X_test = Robust_Scaling(X_train, X_test, normalize_columns)
robust_rfr, y_train_pred, y_test_pred, train_accuarcy = Rfr_model(robust_X_train, robust_X_test, log_y_train, log_y_test)
rfr_feature_importances(robust_rfr, robust_X_train)
trans_y_test, trans_y_test_pred = Exp_y(log_y_test, y_test_pred)
rfr_robust = model_evaluation(trans_y_test, trans_y_test_pred, result_name="rfr_robust")

result_df = pd.concat([rfr_Log, rfr_mm, rfr_sd, rfr_robust], axis=1)
pd.options.display.float_format = '{:.3f}'.format
result_df

22
(69428, 21) (29756, 21)
(69428,) (29756,)
학습 정확도: 0.994512725506549
               Feature  Importance
6  transmission_Manual       0.327
1                 year       0.318
5           engineSize       0.169
0                model       0.061
2              mileage       0.046

LGBM rfr_Log Results
평균 제곱 오차(MSE): 4988830.713
평균 절대 오차(MAE): 1171.032
평균 제곱 오차(MSE): 2233.569
평균 절대비율 오차(MAPE): 7.26
결정 계수(R2): 0.95



KeyboardInterrupt: 

In [ ]:
# engineSize 정규화 제외
df = full2_data.copy()
lable_list = ['model']
onhot_list = ['transmission', 'fuelType', 'carMake']
encoding_df = Encoding(df,lable_list,onhot_list)
X_train, X_test, y_train, y_test = Data_split(encoding_df, 'price')

normalize_columns = ["year" , "mileage", "tax", "mpg"] # 표준 정규화할 변수 리스트

# 로그
log_y_train, log_y_test = Log_Trans(y_train, y_test)
log_rfr, y_train_pred, y_test_pred, train_accuarcy = Rfr_model(X_train, X_test, log_y_train, log_y_test)
rfr_feature_importances(log_rfr, X_train)
trans_y_test, trans_y_test_pred = Exp_y(log_y_test, y_test_pred)
rfr_Log = model_evaluation(trans_y_test, trans_y_test_pred, result_name="rfr_Log")

# 정규화
log_y_train, log_y_test = Log_Trans(y_train, y_test)
mm_X_train, mm_X_test = MinMax_Scaling(X_train, X_test, normalize_columns)
mm_rfr, y_train_pred, y_test_pred, train_accuarcy = Rfr_model(mm_X_train, mm_X_test, log_y_train, log_y_test)
rfr_feature_importances(mm_rfr, mm_X_train)
trans_y_test, trans_y_test_pred = Exp_y(log_y_test, y_test_pred)
rfr_mm = model_evaluation(trans_y_test, trans_y_test_pred, result_name="rfr_mm")

# 표준화
log_y_train, log_y_test = Log_Trans(y_train, y_test)
sd_X_train, sd_X_test = Sd_Scaling(X_train, X_test, normalize_columns)
sd_rfr, y_train_pred, y_test_pred, train_accuarcy = Rfr_model(sd_X_train, sd_X_test, log_y_train, log_y_test)
rfr_feature_importances(sd_rfr, sd_X_train)
trans_y_test, trans_y_test_pred = Exp_y(log_y_test, y_test_pred)
rfr_sd = model_evaluation(trans_y_test, trans_y_test_pred, result_name="rfr_sd")

# 표준 정규화
log_y_train, log_y_test = Log_Trans(y_train, y_test)
robust_X_train, robust_X_test = Robust_Scaling(X_train, X_test, normalize_columns)
robust_rfr, y_train_pred, y_test_pred, train_accuarcy = Rfr_model(robust_X_train, robust_X_test, log_y_train, log_y_test)
rfr_feature_importances(robust_rfr, robust_X_train)
trans_y_test, trans_y_test_pred = Exp_y(log_y_test, y_test_pred)
rfr_robust = model_evaluation(trans_y_test, trans_y_test_pred, result_name="rfr_robust")

result_df = pd.concat([rfr_Log, rfr_mm, rfr_sd, rfr_robust], axis=1)
pd.options.display.float_format = '{:.3f}'.format
result_df

In [ ]:
df['fuelType'].unique()

In [ ]:
# 엔진 사이즈 중 0이며 엔진인경우 제외
df = df[~((df['engineSize'] == 0) & (df['fuelType'].isin(["Petrol", "Diesel"])))]
able_list = ['model']
onhot_list = ['transmission', 'fuelType', 'carMake']
encoding_df = Encoding(df,lable_list,onhot_list)
X_train, X_test, y_train, y_test = Data_split(encoding_df, 'price')

normalize_columns = ["year","mileage", "tax", "mpg", "engineSize"] # 표준 정규화할 변수 리스트

# 로그
log_y_train, log_y_test = Log_Trans(y_train, y_test)
log_rfr, y_train_pred, y_test_pred, train_accuarcy = Rfr_model(X_train, X_test, log_y_train, log_y_test)
rfr_feature_importances(log_rfr, X_train)
trans_y_test, trans_y_test_pred = Exp_y(log_y_test, y_test_pred)
rfr_Log = model_evaluation(trans_y_test, trans_y_test_pred, result_name="rfr_Log")

# 정규화
log_y_train, log_y_test = Log_Trans(y_train, y_test)
mm_X_train, mm_X_test = MinMax_Scaling(X_train, X_test, normalize_columns)
mm_rfr, y_train_pred, y_test_pred, train_accuarcy = Rfr_model(mm_X_train, mm_X_test, log_y_train, log_y_test)
rfr_feature_importances(mm_rfr, mm_X_train)
trans_y_test, trans_y_test_pred = Exp_y(log_y_test, y_test_pred)
rfr_mm = model_evaluation(trans_y_test, trans_y_test_pred, result_name="rfr_mm")

# 표준화
log_y_train, log_y_test = Log_Trans(y_train, y_test)
sd_X_train, sd_X_test = Sd_Scaling(X_train, X_test, normalize_columns)
sd_rfr, y_train_pred, y_test_pred, train_accuarcy = Rfr_model(sd_X_train, sd_X_test, log_y_train, log_y_test)
rfr_feature_importances(sd_rfr, sd_X_train)
trans_y_test, trans_y_test_pred = Exp_y(log_y_test, y_test_pred)
rfr_sd = model_evaluation(trans_y_test, trans_y_test_pred, result_name="rfr_sd")

# 표준 정규화
log_y_train, log_y_test = Log_Trans(y_train, y_test)
robust_X_train, robust_X_test = Robust_Scaling(X_train, X_test, normalize_columns)
robust_rfr, y_train_pred, y_test_pred, train_accuarcy = Rfr_model(robust_X_train, robust_X_test, log_y_train, log_y_test)
rfr_feature_importances(robust_rfr, robust_X_train)
trans_y_test, trans_y_test_pred = Exp_y(log_y_test, y_test_pred)
rfr_robust = model_evaluation(trans_y_test, trans_y_test_pred, result_name="rfr_robust")

result_df = pd.concat([rfr_base, rfr_Log, rfr_mm, rfr_sd, rfr_robust], axis=1)
pd.options.display.float_format = '{:.3f}'.format
result_df

In [ ]:
# 엔진 사이즈 이상치, 연도 이상치, 연도 정규화X
df = df[~((df['engineSize'] == 0) & (df['fuelType'].isin(["Petrol", "Diesel"])))]
df = df[df["year"]>1980]
able_list = ['model']
onhot_list = ['transmission', 'fuelType', 'carMake']
encoding_df = Encoding(df,lable_list,onhot_list)
X_train, X_test, y_train, y_test = Data_split(encoding_df, 'price')

normalize_columns = ["mileage", "tax", "mpg", "engineSize"] # 표준 정규화할 변수 리스트

# 로그
log_y_train, log_y_test = Log_Trans(y_train, y_test)
log_rfr, y_train_pred, y_test_pred, train_accuarcy = Rfr_model(X_train, X_test, log_y_train, log_y_test)
rfr_feature_importances(log_rfr, X_train)
trans_y_test, trans_y_test_pred = Exp_y(log_y_test, y_test_pred)
rfr_Log = model_evaluation(trans_y_test, trans_y_test_pred, result_name="rfr_Log")

# 정규화
log_y_train, log_y_test = Log_Trans(y_train, y_test)
mm_X_train, mm_X_test = MinMax_Scaling(X_train, X_test, normalize_columns)
mm_rfr, y_train_pred, y_test_pred, train_accuarcy = Rfr_model(mm_X_train, mm_X_test, log_y_train, log_y_test)
rfr_feature_importances(mm_rfr, mm_X_train)
log_y_test, log_y_test_pred = Exp_y(log_y_test, y_test_pred)
rfr_mm = model_evaluation(trans_y_test, trans_y_test_pred, result_name="rfr_mm")

# 표준화
log_y_train, log_y_test = Log_Trans(y_train, y_test)
sd_X_train, sd_X_test = Sd_Scaling(X_train, X_test, normalize_columns)
sd_rfr, y_train_pred, y_test_pred, train_accuarcy = Rfr_model(sd_X_train, sd_X_test, log_y_train, log_y_test)
rfr_feature_importances(sd_rfr, sd_X_train)
trans_y_test, trans_y_test_pred = Exp_y(log_y_test, y_test_pred)
rfr_sd = model_evaluation(trans_y_test, trans_y_test_pred, result_name="rfr_sd")

# 표준 정규화
log_y_train, log_y_test = Log_Trans(y_train, y_test)
robust_X_train, robust_X_test = Robust_Scaling(X_train, X_test, normalize_columns)
robust_rfr, y_train_pred, y_test_pred, train_accuarcy = Rfr_model(robust_X_train, robust_X_test, log_y_train, log_y_test)
rfr_feature_importances(robust_rfr, robust_X_train)
trans_y_test, trans_y_test_pred = Exp_y(log_y_test, y_test_pred)
rfr_robust = model_evaluation(trans_y_test, trans_y_test_pred, result_name="rfr_robust")

result_df = pd.concat([rfr_base, rfr_Log, rfr_mm, rfr_sd, rfr_robust], axis=1)
pd.options.display.float_format = '{:.3f}'.format
result_df

In [ ]:
pip install optuna scikit-learn

In [ ]:
import optuna
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error

# 데이터 로드
df = full2_data.copy()
df = df[~((df['engineSize'] == 0) & (df['fuelType'].isin(["Petrol", "Diesel"])))]
df = df[df["year"]>1980]
able_list = ['model']
onhot_list = ['transmission', 'fuelType', 'carMake']
encoding_df = Encoding(df,lable_list,onhot_list)
X_train, X_test, y_train, y_test = Data_split(encoding_df, 'price')

# 데이터 스케일링
normalize_columns = ["mileage", "tax", "mpg","engineSize"] # 표준 정규화할 변수 리스트
log_y_train, log_y_test = Log_Trans(y_train, y_test)
robust_X_train, robust_X_test = Robust_Scaling(X_train, X_test, normalize_columns)


# 목적 함수 정의
def objective(trial):
    # 하이퍼파라미터 범위 설정
    n_estimators = trial.suggest_int("n_estimators", 50, 300) # 생성할 트리수
    max_depth = trial.suggest_int("max_depth", 3, 20) # 최대 트리 깊이
    min_samples_split = trial.suggest_int("min_samples_split", 2, 20) # 노드 분할 최소 샘플 수
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 20) # 리프의 최소 샘풀 수

    # 모델 생성
    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42,
        n_jobs=-1
    )

    # 교차 검증을 통한 평가
    score = cross_val_score(model, X_train, y_train, cv=5, scoring="neg_mean_squared_error", n_jobs=-1)
    # MSE가 낮을수록 좋은데 이를 높을 수록 좋은 의미로 변경하기 위해 음수변환을 함.
    return np.mean(score)

# Optuna 최적화 수행
study = optuna.create_study(direction="maximize")  # score 높을 수록 좋은 방향으로 모델로 최적화
study.optimize(objective, n_trials=50, n_jobs=-1) # n_trials는 Optuna가 하이퍼파라미터를 탐색하는 총 시도 횟수

# 최적 하이퍼파라미터 출력
print("Best hyperparameters:", study.best_params)

# 최적 모델 학습 및 평가
best_params = study.best_params
best_model = RandomForestRegressor(**best_params, random_state=42, n_jobs=-1)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Test MSE:", mse)
# 'n_estimators': 117, 'max_depth': 19, 'min_samples_split': 4, 'min_samples_leaf': 2

In [ ]:
# 최종 모델 학습
# 테스트 5개정도 추출해서 예측값과 실제값 확인하기
df = full2_data.copy()
df = df[~((df['engineSize'] == 0) & (df['fuelType'].isin(["Petrol", "Diesel"])))]
df = df[df["year"]>1980]

# 하이퍼 파라미터
n_estimators = 117
max_depth = 19
min_samples_split = 4
min_samples_leaf = 2

# 전처리
lable_list = ['model']
onhot_list = ['transmission', 'fuelType', 'carMake']
encoding_df = Encoding(df,lable_list,onhot_list)

# 데이터 분할
X_train, X_test, y_train, y_test = Data_split(encoding_df, 'price')

# 데이터 스케일링
normalize_columns = ["mileage", "tax", "mpg"] # 표준 정규화할 변수 리스트
log_y_train, log_y_test = Log_Trans(y_train, y_test)
robust_X_train, robust_X_test = Robust_Scaling(X_train, X_test, normalize_columns)

# 모델 학습
model = RandomForestRegressor(n_estimators = 117,
                              max_depth = 19,
                              min_samples_split = 4,
                              min_samples_leaf = 2,
                              random_state=42)
model.fit(robust_X_train, log_y_train)
rfr_feature_importances(model, X_train)
train_accuarcy = model.score(robust_X_train, log_y_train)
print("학습 정확도:", model.score(robust_X_train, log_y_train))
y_test_pred = model.predict(robust_X_test)

# 예측 및 개별 트리 예측값 수집
tree_predictions = np.array([tree.predict(robust_X_test.to_numpy()) for tree in model.estimators_])
"""랜덤포레스트는 feature name을 사용하지만 결정트리에서는 사용하지 않아 오류발생
robust_X_test의 feature name을 제거하기 위해 nupy 배열로 변경"""
tree_predictions = np.expm1(tree_predictions)

# 평균 및 신뢰구간 계산
y_mean = tree_predictions.mean(axis=0)
y_std = tree_predictions.std(axis=0)
lower_bound = y_mean - 1.96 * y_std  # 95% 신뢰구간 하한
upper_bound = y_mean + 1.96 * y_std  # 95% 신뢰구간 상한


# 모델 평가
trans_y_test, trans_y_test_pred = Exp_y(log_y_test, y_test_pred)
Last_result = model_evaluation(trans_y_test, trans_y_test_pred, result_name="Last_result")
print(Last_result)

# 신뢰구간 그래프 시각화
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.scatter(range(len(trans_y_test)), trans_y_test, color='blue', label="Actual")
plt.scatter(range(len(trans_y_test_pred)), trans_y_test_pred, color='red', label="Predicted")
plt.fill_between(range(len(trans_y_test_pred)), trans_lower_bound, trans_upper_bound, color='gray', alpha=0.3, label="95% CI")
plt.legend()
plt.xlabel("Sample Index")
plt.ylabel("Price")
plt.title("Predicted vs Actual with 95% Confidence Interval")
plt.show()

# 테스트 데이터 일부 샘플링
np.random.seed(42)
sample_indices = np.random.choice(len(X_test), size=5, replace=False)
X_sample = X_test.iloc[sample_indices]
y_sample = y_test.iloc[sample_indices]
# 셈플 신뢰구간 구하기
tree_predictions = np.array([tree.predict(X_sample.to_numpy()) for tree in model.estimators_])
tree_predictions = np.expm1(tree_predictions)
y_sample_mean = tree_predictions.mean(axis=0)
y_sample_std = tree_predictions.std(axis=0)
y_sample_lower_bound = y_sample_mean - 1.96 * y_sample_std
y_sample_upper_bound = y_sample_mean + 1.96 * y_sample_std

#결과 출력
result_df = X_sample.copy()
encoding_df=df.copy()
le  = LabelEncoder()
encoding_df[lable_list] = encoding_df[lable_list].apply(le.fit_transform)
result_df[lable_list] = result_df[lable_list].apply(le.inverse_transform)
for category in onhot_list:
    category_columns = [col for col in result_df.columns if col.startswith(category + "_")]
    result_df[category] = result_df[category_columns].idxmax(axis=1).str.replace(category + "_", "")
    result_df.drop(columns=category_columns, inplace=True)
result_df["Actual"] = y_sample
result_df["Predicted"] = y_sample_mean
result_df["Lower Bound (95%)"] = y_sample_lower_bound
result_df["Upper Bound (95%)"] = y_sample_upper_bound
print(result_df)

# 결과 시각화
plt.figure(figsize=(10, 5))
plt.errorbar(range(5), y_sample_mean,
             yerr=1.96 * y_sample_std, fmt='o', label="Predicted (95% CI)", color='red')
plt.scatter(range(5), y_sample, color='blue', label="Actual")
plt.xlabel("Sample Index")
plt.xticks([0,1,2,3,4])
plt.ylabel("Price")
plt.legend()
plt.title("Random Forest Regression: Prediction with 95% Confidence Interval")
plt.show()

result_df